In [ ]:
import re
import shutil
from pathlib import Path

import branca.colormap as cm
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import rasterio.mask
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
from pyproj import Transformer
from rasterio.plot import show
from rasterio.warp import Resampling, calculate_default_transform, reproject
from shapely.geometry import shape
from pylandtemp import single_window

### Paths

In [ ]:
# ID of the Landsat 8 files
id = "LC08_L1TP_193023_20220803_20220806_02_T1"

In [ ]:
# Directory paths
dir_data_root = Path("../data/landsat/")
dir_files = Path(dir_data_root, id)
dir_output = Path(dir_data_root, "output")

# File paths
geojson_path = Path(dir_data_root, "geojson", "berlin.geojson")
mtl_path = Path(dir_files, id + "_MTL.txt")

# Bands
band_4_path = Path(dir_files, id + "_B4.TIF")
band_5_path = Path(dir_files, id + "_B5.TIF")
band_10_path = Path(dir_files, id + "_B10.TIF")

# Paths clipped to GeoJson
band_4_clipped_path = clip_to_geojson(band_4_path, geojson_path, dir_output)
band_5_clipped_path = clip_to_geojson(band_5_path, geojson_path, dir_output)
band_10_clipped_path = clip_to_geojson(band_10_path, geojson_path, dir_output)
# TODO: Somehow clip_to_geojson() deletes MTL file

# Output file paths
file_temperature = Path(dir_output, "land_surface_temperature.tif")
file_temperature_repr = Path(dir_output, "temperature_reprojected.tif")
file_temperature_repr_colored = Path(dir_output, "temperature_reprojected_colored.tif")

### Functions

In [ ]:
def calc_lst(band_4_path, band_5_path, band_10_path, target_path):
    """
    Calculates the land surface temperature (LST) from Landsat 8
    bands 4, 5 and 10 using pylandtemp library.
    """
    with rasterio.open(band_4_path) as src:
        band_4 = src.read(1)

    with rasterio.open(band_5_path) as src:
        band_5 = src.read(1)

    with rasterio.open(band_10_path) as src:
        band_10 = src.read(1)
        out_meta = src.meta.copy()

    lst_image_array = single_window(band_10, band_4, band_5, unit="celsius")
    
    # For some reason, the values are in Kelvin, so we need to convert them to Celsius
    lst_image_array = lst_image_array - 273.15

    out_meta.update(
        {
            "height": lst_image_array.shape[0],
            "width": lst_image_array.shape[1],
            "transform": src.transform,
            "dtype": lst_image_array.dtype,
        }
    )

    with rasterio.open(target_path, "w", **out_meta) as dst:
        dst.write(lst_image_array, 1)

    print("Saved Land Surface Temperature (LST) to", target_path)

    return lst_image_array

In [ ]:
def exaggerate(temp_array, factor=2):
    """
    Exaggerate the values of an array.
    """
    # Calculate the mean temperature
    mean_temp = np.mean(temp_array)

    valid_mask = (temp_array != 0) & (temp_array > mean_temp)

    deviation = np.where(valid_mask, temp_array - mean_temp, 0)

    # Calculate the deviation from the mean
    # deviation = temp_array - mean_temp

    # Apply an exaggeration function to the deviation
    exaggerated_deviation = np.power(deviation, factor)

    # Add the exaggerated deviation back to the mean temperature
    exaggerated_temperature = mean_temp + exaggerated_deviation

    return exaggerated_temperature

In [ ]:
def clip_to_geojson(band_path, geojson_path, target_dir):
    """
    Clip a GeoTIFF file to an Area of Interest (AOI) from a Geojson file.
    """
    # Load a GeoJSON or shapefile of the area of interest
    aoi = gpd.read_file(geojson_path)

    # Set correct CRS (obtained from:
    # check = rasterio.open(band_path)
    # check.crs
    aoi = aoi.to_crs("EPSG:32633")  # TODO Take directly from src.crs

    # Transform to a Shapely geometry
    aoi_shape = [shape(aoi.geometry.loc[0])]

    # Open the Landsat band file
    with rasterio.open(band_path) as src:
        # Clip the raster file with the polygon using mask function
        out_image, out_transform = rasterio.mask.mask(src, aoi_shape, crop=True)
        out_meta = src.meta

    # Update the metadata to include the new transform and shape
    out_meta.update(
        {
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
        }
    )

    file_path_new = target_dir / f"{band_path.stem}_clipped{band_path.suffix}"

    # Save the clipped raster to a new GeoTiff file
    with rasterio.open(file_path_new, "w", **out_meta) as dest:
        dest.write(out_image)

    print(f"Saved clipped file to {file_path_new}")

    return str(file_path_new)

#### Prepare files for Folium layer

In [ ]:
def convert_values_to_colors(values, cmap):
    """
    Convert image values to colors using a colormap.
    
    Inputs:
        - values: 2D NumPy array of image values
        - cmap: a branca.colormap.LinearColormap object
    Output:
        - 3D NumPy array representing colored image
    """
    colors_arr = np.zeros((*values.shape, 4))
    for i in range(values.shape[0]):
        for j in range(values.shape[1]):
            value = values[i, j]
            if np.isnan(value):
                colors_arr[i, j, :] = [0, 0, 0, 0]  # Transparent color for NaN values
            else:
                color = colors.to_rgba(cmap(value), alpha=0.7)
                colors_arr[i, j, :] = color
    return colors_arr

In [ ]:
def reproject_geotiff(src_path, target_path, target_crs):
    """
    Function to reproject a GeoTiff to a different CRS.
    """
    # Open the raster file
    with rasterio.open(src_path) as src:
        img = src.read().astype(np.float32)

        # Set the nodata values to NaN
        img[img == src.nodata] = np.nan

        # Define target CRS (e.g., WGS84)
        target_crs = rasterio.crs.CRS.from_epsg(target_crs)

        # Calculate the default transform for the reprojected image
        transform, width, height = rasterio.warp.calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )

        # Create an array to hold the reprojected image
        reprojected_img = np.zeros((img.shape[0], height, width), dtype=img.dtype)

        # Reproject the image to match the transformed bounds
        reproject(
            src.read(),
            reprojected_img,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=target_crs,
        )

        # Update the metadata for the reprojected image
        reprojected_meta = src.meta.copy()
        reprojected_meta.update(
            {
                "crs": target_crs,
                "transform": transform,
                "width": width,
                "height": height,
            }
        )

        # Set the nodata values to NaN
        reprojected_img[reprojected_img == src.nodata] = np.nan

        # Write the reprojected image to a new GeoTIFF file
        with rasterio.open(target_path, "w", **reprojected_meta) as dst:
            dst.write(reprojected_img)

        return dst, reprojected_img

In [ ]:
def create_rgba_color_image(src_path, target_path):
    """
    Function to map raster values to rgba.
    """
    with rasterio.open(src_path) as src:
        img = src.read()#.astype(np.float32)

        # Set the nodata values to NaN
        #img[img == src.nodata] = np.nan
        
        vmin = np.floor(np.nanmin(img))
        vmax = np.ceil(np.nanmax(img))

        colormap = cm.linear.RdBu_11.scale(vmin, vmax)
        colormap.colors.reverse()

        colored_img = convert_values_to_colors(img[0], colormap)

        # Update src.meta
        colored_meta = src.meta.copy()
        colored_meta.update({"count": 4})

        # Write the reprojected image to a new GeoTIFF file
        with rasterio.open(target_path, "w", **colored_meta) as dst:
            # Move channel information to first axis
            colored_img_rio = np.moveaxis(colored_img, source=2, destination=0)
            dst.write(colored_img_rio)

        return dst, colored_img

### Calculate metrics

In [ ]:
lst = calc_lst(
    band_4_clipped_path, band_5_clipped_path, band_10_clipped_path, file_temperature
)

In [ ]:
plt.imshow(lst, cmap="RdYlBu_r")
plt.colorbar(label="Land surface temperature (°C)")
plt.title("Land surface temperature (°C)")
plt.show()

### Change colors and projection

In [ ]:
reprojected_img, reprojected_img_array = reproject_geotiff(
    file_temperature, file_temperature_repr, 4326
)

In [ ]:
with rasterio.open(file_temperature_repr) as src:
    temperature_reprojected = src.read(1)

plt.imshow(temperature_reprojected, cmap="RdYlBu_r")
plt.colorbar(label="Land surface temperature (°C)")
plt.title("Land surface temperature (°C)")

In [ ]:
colored_img, colored_img_array = create_rgba_color_image(
    reprojected_img.name, file_temperature_repr_colored
)

In [ ]:
with rasterio.open(file_temperature_repr_colored) as src:
    temperature_reprojected_colored = src.read(1)

plt.imshow(temperature_reprojected_colored, cmap="RdYlBu_r");

### Plot temperature map

In [ ]:
with rasterio.open(file_temperature_repr_colored) as colored_img:
    # Load GeoTIFF as ndarray
    colored_img_array = colored_img.read()

    # Move channel axis to third position
    colored_img_array = np.moveaxis(colored_img_array, source=0, destination=2)

In [ ]:
m = folium.Map(
    location=[
        (colored_img.bounds.bottom + colored_img.bounds.top) / 2,
        (colored_img.bounds.left + colored_img.bounds.right) / 2,
    ],
    tiles="Stamen Terrain",
    zoom_start=11,
)
folium.raster_layers.ImageOverlay(
    image=colored_img_array,
    name="Land Surface Temperature",
    opacity=0.75,
    bounds=[
        [colored_img.bounds.bottom, colored_img.bounds.left],
        [colored_img.bounds.top, colored_img.bounds.right],
    ],
).add_to(m)

folium.LayerControl().add_to(m)

# colormap.caption = "Surface temperature"
# m.add_child(colormap)

# Set bounds TODO: Not working
# m.fit_bounds(
#     [
#         [colored_img.bounds.bottom, colored_img.bounds.left],
#         [colored_img.bounds.top, colored_img.bounds.right],
#     ]
# )

m

### Unused functions

In [ ]:
def load_geotiff(file_path, target_crs=None):
    # Open the GeoTIF file
    with rasterio.open(file_path) as src:
        # Read the raster data as a numpy ndarray
        raster = src.read(1).astype(np.float32)

    if target_crs is not None:
        # Get the current CRS (Coordinate Reference System)
        crs = src.crs

        # Calculate the transformation parameters for reprojecting
        transform, width, height = calculate_default_transform(
            crs, target_crs, src.width, src.height, *src.bounds
        )

        # Create an array to store the reprojected raster
        reprojected_raster = np.zeros((height, width), dtype=np.float32)

        # Reproject the raster to Mercator
        reproject(
            raster,
            reprojected_raster,
            src_transform=src.transform,
            src_crs=crs,
            dst_transform=transform,
            dst_crs=target_crs,
            resampling=Resampling.nearest,
        )

        return reprojected_raster

    return raster

In [ ]:
def load_raster(path, projection=None):
    """
    Function to load raster files into numpy arrays.
    """

    # Open the file:
    with rasterio.open(path) as src:
        # Read the grid values into numpy arrays
        r = src.read(1)
        # Convert to numpy arrays
        r = r.astype(np.float32)
        # Convert NoData to NaN
        r[r == src.nodata] = np.nan

    # Convert to projection
    if projection is not None:
        r = rasterio.reproject(r, src.crs, projection)

    return r

### 3D plot

In [ ]:
# Create a grid of x and y values
y = np.linspace(0, temperature.shape[0], temperature.shape[0])
x = np.linspace(0, temperature.shape[1], temperature.shape[1])
x, y = np.meshgrid(x, y)

# Create a new matplotlib figure and 3D axes
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(x, y, temperature, cmap="RdYlBu_r", linewidth=0, antialiased=False)

# Add a color bar which maps values to colors
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
temperature_exaggerated = exaggerate(temperature, factor=3)

In [ ]:
plt.hist(temperature_exaggerated.flatten(), bins=200);

In [ ]:
# Create a grid of x and y values
y = np.linspace(0, temperature_exaggerated.shape[0], temperature_exaggerated.shape[0])
x = np.linspace(0, temperature_exaggerated.shape[1], temperature_exaggerated.shape[1])
x, y = np.meshgrid(x, y)

# Create a new matplotlib figure and 3D axes
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(x, y, temperature_exaggerated, cmap='Reds', linewidth=0, antialiased=False)

# Add a color bar which maps values to colors
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

### Visualize truecolor image from level 2 data

In [ ]:
path_level_2 = "data/LC09_L2SP_193023_20220811_20230403_02_T1/"

In [ ]:
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)


# Define the bands
red_band = path_level_2 + "LC09_L2SP_193023_20220811_20230403_02_T1_SR_B2.TIF"
green_band = path_level_2 + "LC09_L2SP_193023_20220811_20230403_02_T1_SR_B3.TIF"
blue_band = path_level_2 + "LC09_L2SP_193023_20220811_20230403_02_T1_SR_B4.TIF"

# Open and read each band using rasterio
with rasterio.open(red_band) as src:
    red = src.read(1)

with rasterio.open(green_band) as src:
    green = src.read(1)

with rasterio.open(blue_band) as src:
    blue = src.read(1)

# Normalize each band to 0 - 1 scale
red_norm = normalize(red)
green_norm = normalize(green)
blue_norm = normalize(blue)

# Stack bands
nrg = np.dstack((red_norm, green_norm, blue_norm))

# View the color composite
plt.imshow(nrg)
plt.show()

In [ ]:
def linear_stretch(input_array, percent):
    """Performs a linear stretch of the input array values"""
    p_low, p_high = np.percentile(input_array[~np.isnan(input_array)], (percent, 100 - percent))
    return (input_array - p_low) / (p_high - p_low)
  
# Define the stretch percentage
percent = 2

# Apply the stretch to each band
red_stretched = linear_stretch(red_norm, percent)
green_stretched = linear_stretch(green_norm, percent)
blue_stretched = linear_stretch(blue_norm, percent)

# Stack bands
rgb_stretched = np.dstack((red_stretched, green_stretched, blue_stretched))

# Clip the values outside the 0-1 range
rgb_stretched = np.clip(rgb_stretched, 0, 1)

# View the color composite
plt.imshow(rgb_stretched)
plt.show()

In [ ]:
import rasterio.mask

# Load a GeoJSON or shapefile of the area of interest
aoi = gpd.read_file("data/berlin.geojson")

# Set correct CRS
aoi = aoi.to_crs('EPSG:32633')

# Transform to a Shapely geometry
aoi_shape = [shape(aoi.geometry.loc[0])]

# Read the RGB bands, apply the mask, and squeeze the singleton dimensions
with rasterio.open(red_band) as src:
    out_image_red, out_transform = rasterio.mask.mask(src, aoi_shape, crop=True)
    out_image_red = np.squeeze(out_image_red)
    out_meta = src.meta

with rasterio.open(green_band) as src:
    out_image_green, _ = rasterio.mask.mask(src, aoi_shape, crop=True)
    out_image_green = np.squeeze(out_image_green)

with rasterio.open(blue_band) as src:
    out_image_blue, _ = rasterio.mask.mask(src, aoi_shape, crop=True)
    out_image_blue = np.squeeze(out_image_blue)

# Normalize each band to 0 - 1 scale and stretch the contrast
red_stretched = linear_stretch(normalize(out_image_red), percent)
green_stretched = linear_stretch(normalize(out_image_green), percent)
blue_stretched = linear_stretch(normalize(out_image_blue), percent)

# Stack bands
rgb_stretched = np.dstack((red_stretched, green_stretched, blue_stretched))

# Clip the values outside the 0-1 range
rgb_stretched = np.clip(rgb_stretched, 0, 1)

# Reorder the dimensions for the plot
rgb_stretched_plot = np.transpose(rgb_stretched, (1, 2, 0))

# View the color composite
rgb_stretched_plot = np.transpose(rgb_stretched, (0, 1, 2))
plt.imshow(rgb_stretched_plot)
plt.show()

# Convert to 8-bit unsigned integer (0-255)
rgb_8bit = (rgb_stretched * 255).astype(np.uint8)

# Update metadata
out_meta.update({"driver": "GTiff",
                 "height": rgb_8bit.shape[0],
                 "width": rgb_8bit.shape[1],
                 "transform": out_transform,
                 "dtype": 'uint8',
                 "count": 3})

# Reorder the dimensions for writing
rgb_8bit = np.transpose(rgb_8bit, (2, 0, 1))

# Write the clipped and stretched RGB image to a new TIFF file
with rasterio.open("output.tif", "w", **out_meta) as dest:
    dest.write(rgb_8bit)

In [ ]:
# Load a GeoJSON or shapefile of the area of interest
aoi = gpd.read_file("data/berlin.geojson")

# Set correct CRS
aoi = aoi.to_crs('EPSG:32633')

# Transform to a Shapely geometry
aoi_shape = [shape(aoi.geometry.loc[0])]

# Read the RGB bands, apply the mask, and squeeze the singleton dimensions
with rasterio.open(red_band) as src:
    out_image_red, out_transform = rasterio.mask.mask(src, aoi_shape, crop=True)
    out_image_red = np.squeeze(out_image_red)
    out_meta = src.meta

with rasterio.open(green_band) as src:
    out_image_green, _ = rasterio.mask.mask(src, aoi_shape, crop=True)
    out_image_green = np.squeeze(out_image_green)

with rasterio.open(blue_band) as src:
    out_image_blue, _ = rasterio.mask.mask(src, aoi_shape, crop=True)
    out_image_blue = np.squeeze(out_image_blue)

# Normalize each band to 0 - 1 scale and stretch the contrast
red_stretched = normalize(out_image_red)
green_stretched = normalize(out_image_green)
blue_stretched = normalize(out_image_blue)

# Stack bands
rgb_stretched = np.dstack((red_stretched, green_stretched, blue_stretched))

# Clip the values outside the 0-1 range
rgb_stretched = np.clip(rgb_stretched, 0, 1)

# Reorder the dimensions for the plot
rgb_stretched_plot = np.transpose(rgb_stretched, (0, 1, 2))

# View the color composite
plt.imshow(rgb_stretched_plot)
plt.show()

# Update metadata
out_meta.update({"driver": "GTiff",
                 "height": rgb_stretched.shape[0],
                 "width": rgb_stretched.shape[1],
                 "transform": out_transform,
                 "dtype": 'float32',
                 "count": 3})

# Reorder the dimensions for writing
rgb_stretched = np.transpose(rgb_stretched, (2, 0, 1))

# Write the clipped and stretched RGB image to a new TIFF file
with rasterio.open("output.tif", "w", **out_meta) as dest:
    dest.write(rgb_stretched)


In [ ]:
check = rasterio.open(band_10_path)
check.crs

In [ ]:
import numpy as np
import folium
from folium.plugins import HeatMap
from rasterio.warp import transform
from pyproj import CRS, Transformer

# Load the Landsat data and transformation matrix
# This code assumes that you have a variable `temperature_C` containing the temperature data
# and a variable `affine_trans` containing the affine transformation matrix from your Landsat file

# Replace 'raster_file.tif' with the path to your raster file
with rasterio.open(band_10_path) as ds:
    affine_trans = ds.transform

# Create arrays for pixel coordinates
rows, cols = np.indices((temperature.shape))
x_geo, y_geo = rasterio.transform.xy(affine_trans, rows.flatten(), cols.flatten())

# Create a transformer to convert from UTM to lat/long
source_crs = CRS.from_epsg(32633)  # replace with the EPSG code of your raster's CRS
target_crs = CRS.from_epsg(4326)
transformer = Transformer.from_crs(source_crs, target_crs)

# Convert the geographic coordinates to lat/long
latitudes, longitudes = transformer.transform(y_geo, x_geo)

# Define a gradient with transparency
gradient = {0.4: 'rgba(0,255,0,0.5)', 0.65: 'rgba(255,255,0,0.5)', 1: 'rgba(255,0,0,0.5)'}

# Combine the latitudes, longitudes, and temperatures into a single array
data = np.array([latitudes, longitudes, temperature.flatten()]).T

# Create a folium map centered on the middle of the dataset
m = folium.Map(location=[np.mean(latitudes), np.mean(longitudes)], zoom_start=13)

# Add a heatmap layer
HeatMap(data, gradient=gradient, radius=8, max_zoom=13).add_to(m)

# Display the map
m
